In [1]:
import sys
sys.path.insert(0, '../../toyaikit/')

In [2]:
%load_ext autoreload
%autoreload 2

In [108]:
from toyaikit.utils import strip_matching_outer_html_tags

In [3]:
from openai import OpenAI

In [4]:
from toyaikit.tools import Tools
from toyaikit.chat import IPythonChatInterface
from toyaikit.llm import OpenAIClient
from toyaikit.chat.runners import OpenAIResponsesRunner, DisplayingRunnerCallback

In [5]:
import os
import shutil

def start(project_name):
    if not project_name:
        print("Project name cannot be empty.")
        return False

    if os.path.exists(project_name):
        print(f"Directory '{project_name}' already exists. Please choose a different name or remove the existing directory.")
        return False

    shutil.copytree('django_template', project_name)
    print(f"Django template copied to '{project_name}' directory.")

    return True

In [6]:
chat_interface = IPythonChatInterface()
openai_client = OpenAIClient(client=OpenAI())

In [74]:
from pydantic import BaseModel
from typing import Literal

In [117]:
agent_0_prompt = """
You are an AI coding agent tasked with transforming a single user prompt into a well-defined set
of functional requirements for a Django application.

Your Responsibilities:

1. Interpret the user’s intent and extract clear, actionable functional requirements.
2. Avoid technical implementation details (e.g., frameworks, libraries, or architecture). 
   Focus solely on what the application should do, not how.
3. Justify your choices — briefly explain the reasoning behind your interpretation and naming decisions.
4. Focus on the most essential functionality. Think what is really necessary for the MVP and include only this.
5. Make sure that all the essential functionality is available and accessible on the main page of the website.
6. Name the project with a unique, meaningful title that reflects its purpose and avoids naming conflicts.
7. Generate a slug (a URL- and filesystem-safe version of the project name) to serve as the project folder name.

Your output will be passed to the technical planning agent, so clarity and precision are critical.

You can ask the user clarifying questions before coming up with the final set of requirements.
Ask one question at the time, not to overwhelm the user. Make each question short and easy to understand,
ideally with yes/no answers, or something that requires a very short answer.

Limit it to 3 most essential questions. In this case, just ask the question without following any format.

When you're ready with the complete list of the requirement, format the result according to this template: 

<OUTPUT>
REQUIREMENTS_READY

Here include the description of the project - its main functionality
and other important things

1. Requirement 1
   Detailed explanation
2. Requirement 2
   Detailed explanation
...
N. Requirement N
   Detailed explanation
</OUTPUT>
""".strip()

In [118]:
llm_client = OpenAIClient(
    client=OpenAI(),
    model='gpt-5-nano',
    extra_kwargs={'reasoning': {"effort": "low"}}
)

agent_0 = OpenAIResponsesRunner(
    tools=None,
    developer_prompt=agent_0_prompt,
    chat_interface=chat_interface,
    llm_client=llm_client
)

In [119]:
def stop_on_ready(messages):
    if len(messages) == 0:
        return False

    entry = messages[-1]
    if entry.type == 'message':
        text = entry.content[0].text
        if 'REQUIREMENTS_READY' in text:
            return True

    return False

In [120]:
messages = agent_0.run(stop_criteria=stop_on_ready)

You: todo list


You: yes


You: no


You: yes


In [127]:
requirements = strip_matching_outer_html_tags(messages[-1].content[0].text)
requirements = requirements.replace('REQUIREMENTS_READY', '').strip()

In [130]:
name_agent = OpenAIResponsesRunner(
    developer_prompt="extract or create slug for the project based on the requirements, don't include anything else",
    llm_client=OpenAIClient(model='gpt-4o-mini', client=OpenAI())
)

name_response = name_agent.loop(prompt=requirements)

In [131]:
slug = name_response[-1].content[0].text
print(slug)

novatodo


In [132]:
print(slug)
print()
print()
print(requirements)

novatodo


Project title: NovaTodo
Slug: novatodo

Reasoning: NovaTodo conveys a modern, clear todo-list application with user-specific data. The slug novatodo is concise and filesystem/URL friendly.

Main functionality:
- Authenticated user experience: Users must sign up, log in, and log out. Each user has a private set of tasks visible only to them.
- Core task management from the main page: On the main page, a user can create new tasks, edit existing tasks, delete tasks, and mark tasks as complete, all from a single, central view.
- Task data model (essential fields): Each task has a required title and optional details/notes. A boolean completed flag tracks completion. All tasks are associated with their owning user. Timestamps for creation and last update are stored for reference.

Detailed requirements:
1. User authentication
   - Users can create an account with a username/email and password.
   - Users can sign in and sign out.
   - Access to the main task view requires authenti

In [133]:
start(slug)

Django template copied to 'novatodo' directory.


True

In [134]:
from pathlib import Path
import tools

project_path = Path(slug)
agent_tools = tools.AgentTools(project_path)

In [137]:
agent_1_prompt = """
You are a planning agent responsible for designing the application based on functional requirements.
You assist the coding agent by generating a detailed, actionable implementation plan.

Your Role:

- You get a set of functional requirements from the requirements agent.
- You do not modify the codebase directly, but you must describe precisely the changes
  and actions that should be taken.
- Your goal is to translate functional requirements into a clear step-by-step plan for
  how to implement the desired functionality in the existing Django project.

Instructions:

- Check the file structure to better plan your work
- Always include styling in the plan.
- For Django forms, add custom stylings to styles.css. Make sure forms have styles, otherwise they are not visible. 
- Avoid complex logic in templates — move data processing to the server side (views or models) whenever possible.
- Make sure the main interaction elements are accessible from the home page
- Focus on clarity, modularity, and maintainability in your plan.
- Don't include the exact code in the output, focus on the instructions.

Your output will be consumed by a coding agent, so it must be precise, unambiguous,
and broken down into logical steps.

Output template format:

<OUTPUT>
# Project name

Project details

[ ] Step 1
    Detailed description
[ ] Step 2
    Detailed description
... 
[ ] Step N
    Detailed description
</OUTPUT>
"""

In [138]:
llm_client_1 = OpenAIClient(
    client=OpenAI(),
    model='gpt-5-nano',
    extra_kwargs={'reasoning': {"effort": "low"}}
)

agent_1_tools = Tools()
agent_1_tools.add_tool(agent_tools.see_file_tree)

agent_1 = OpenAIResponsesRunner(
    tools=agent_1_tools,
    developer_prompt=agent_1_prompt,
    chat_interface=chat_interface,
    llm_client=llm_client_1
)

In [139]:
result1 = agent_1.loop(
    prompt=requirements,
)

In [140]:
execution_plan = result1[-1].content[0].text
execution_plan = strip_matching_outer_html_tags(execution_plan)

In [141]:
print(execution_plan)

# NovaTodo

Project details
- Django-based TODO app with per-user task management.
- MVP focuses on authentication, per-user CRUD for tasks, and inline actions on the main page.
- Styling is applied via static CSS to ensure forms and UI elements are visible and accessible.
- All actions occur on the main page with appropriate security and validation.

[ ] Step 1: Assess and align project structure
    - Inspect the repository to confirm existing Django project layout (settings, urls, wsgi/asgi, static and media settings).
    - Identify the app(s) to host the task functionality (create a new app if none exists, e.g., "tasks" or "todo").
    - Ensure Django templates, static files (CSS), and static file collection are properly configured.
    - Verify that the project uses Django's authentication system (User model) and that the project includes the login, logout, and signup flows or plan to implement them.

[ ] Step 2: Define data model (Task)
    - Create a new model Task with the fol

In [142]:
DEVELOPER_PROMPT = """
You are a coding agent responsible for applying specific modifications to
a Django project based on instructions from a planning agent.

You are given a plan and you need to execute it step by step. 
Follow the instructions precisely and modify the codebase directly
using the tools available to you.

Each time you execute a step, give a short explanation of what you did and which
step number you finished.

You do not ask questions. You do not suggest. You execute.

All changes must adhere to Django best practices and fit cleanly into the existing project structure.

## Project Context

This is a Django 5.2.4 web application scaffolded with common conventions and clean separation of concerns.

Key technologies and constraints:

- Python 3.8+
- Django 5.2.4
- SQLite for local database
- TailwindCSS for all styling
- HTML templates used for rendering

## Directory Structure

├── README.md  
├── manage.py  
├── pyproject.toml  
├── uv.lock  
├── myapp/  
│   ├── __init__.py  
│   ├── admin.py  
│   ├── apps.py  
│   ├── migrations/  
│   │   └── __init__.py  
│   ├── models.py  
│   ├── templates/  
│   │   └── home.html  
│   ├── tests.py  
│   └── views.py  
├── myproject/  
│   ├── __init__.py  
│   ├── asgi.py  
│   ├── settings.py  
│   ├── urls.py  
│   └── wsgi.py  
├── myproject/  
├── static
|   └── css
|       └── styles.css
└── templates/  
    └── base.html

## File Notes

- `manage.py`: Entry point for Django commands
- `myapp/`: Main application logic, templates, and data models
- `myproject/`: Project-wide configuration and routing
- `templates/`: Shared layout templates (e.g., base.html)
- `static/css/styles.css`: Custom styling, especially important for forms

## Execution rules

- Do not run the development server (e.g. runproject), but you may run diagnostic or validation commands.
- Use TailwindCSS to create clean and user-friendly UI components. Avoid creating custom styles unless really necessary.
- Include emojis and pictograms where it enhances the user interface. Font Awesome is available.
- Do not place logic in templates. Use views or models for data processing and transformation.
- You can create, edit, or delete any files as needed to complete your task.
- When adding forms, make sure you add custom styling to the styles.css.
  Forms typically look like that when rendered (some things omitted for brevity):
```html
<form method="post">
  <p>
    <label for="id_title">Title:</label>
    <input type="text" id="id_title">
  </p>
  <p>
    <label for="id_description">Description:</label>
    <textarea name="description" cols="40" rows="10" id="id_description"></textarea>
  </p>
  <p>
    <label for="id_status">Status:</label>
    <select name="status" id="id_status">
      <option value="pending" selected="">Pending</option>
      <option value="completed">Completed</option>
    </select>
  </p>
  <button type="submit" class="btn btn-primary">Create</button>
</form>
```

After execution is complete, show a very short summary and describe how to run the
application and access the files.

Act with precision. Implement the plan.
"""

In [143]:
tools_obj = Tools()
tools_obj.add_tools(agent_tools)

In [144]:
from toyaikit.llm import OpenAIChatCompletionsClient
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIChatCompletionsRunner

In [145]:
zai_client = OpenAI(
    api_key=os.getenv('ZAI_API_KEY'),
    base_url='https://api.z.ai/api/paas/v4/'
)

zai_llm_client = OpenAIChatCompletionsClient(
    model='glm-4.5',
    client=zai_client
)

In [146]:
coding_agent = OpenAIChatCompletionsRunner(
    tools=tools_obj,
    developer_prompt=DEVELOPER_PROMPT,
    chat_interface=chat_interface,
    llm_client=zai_llm_client
)

In [148]:
full_plan = f"""
{execution_plan}

## Extra steps:

[ ] Extra Step 1: Make sure TailwindCSS is used and the URL is included in the base template
[ ] Extra Step 2: If we used Django forms, make sure that there are styles in styles.css for them
[ ] Extra Step 3: Make migrations and migrate
[ ] Extra Step 4: Add tests to make sure all templates render correctly and execute them
""".strip()

displaying_callback = DisplayingRunnerCallback(chat_interface)
results = coding_agent.loop(prompt=full_plan, callback=displaying_callback)

In [149]:
git_diff, stderr, _ = agent_tools.execute_bash_command('git diff')

In [150]:
print(git_diff)

diff --git a/README.md b/README.md
index 3566040..901240b 100644
--- a/README.md
+++ b/README.md
@@ -1,27 +1,207 @@
-# Running the Django App
+# NovaTodo - Django Task Management App
 
-This project uses [uv](https://github.com/astral-sh/uv) for Python environment and dependency management, and Django as the web framework.
+A modern, user-friendly task management application built with Django 5.2.4, featuring per-user task isolation, inline CRUD operations, and responsive design.
 
-## Prerequisites
+## Features
 
-- Python 3.8+
-- [uv](https://github.com/astral-sh/uv) installed globally (`pip install uv`)
+- 🔐 **User Authentication**: Complete signup, login, and logout functionality
+- 📝 **Task Management**: Create, read, update, and delete tasks
+- ✅ **Task Completion**: Toggle task status with visual feedback
+- 🔒 **User Isolation**: Each user can only see and manage their own tasks
+- 📱 **Responsive Design**: Works seamlessly on desktop and mobile devices
+- 🎨 **Modern UI**: Clean 

In [47]:
prompt = f'''

Continue with execution

Initial plan: 

{full_plan}

Changes made so far:

{git_diff}
'''

results = coding_agent.loop(prompt=prompt, callback=displaying_callback)

In [152]:
agent_tools.execute_bash_command("git commit -am v1")

('[main ece4c90] v1\n 10 files changed, 1016 insertions(+), 118 deletions(-)\n rewrite README.md (88%)\n rewrite myapp/templates/home.html (80%)\n rewrite myapp/tests.py (98%)\n rewrite myapp/views.py (83%)\n rewrite static/css/styles.css (65%)\n',
 'warning: CRLF will be replaced by LF in README.md.\nThe file will have its original line endings in your working directory\nwarning: CRLF will be replaced by LF in myapp/admin.py.\nThe file will have its original line endings in your working directory\nwarning: CRLF will be replaced by LF in myapp/models.py.\nThe file will have its original line endings in your working directory\nwarning: CRLF will be replaced by LF in myapp/templates/home.html.\nThe file will have its original line endings in your working directory\nwarning: CRLF will be replaced by LF in myapp/tests.py.\nThe file will have its original line endings in your working directory\nwarning: CRLF will be replaced by LF in myapp/views.py.\nThe file will have its original line end

In [154]:
results = coding_agent.loop(prompt="make forms on the login page visible by adding styling", callback=displaying_callback)

In [155]:
git_diff, stderr, _ = agent_tools.execute_bash_command('git diff')

In [156]:
print(git_diff)

diff --git a/static/css/styles.css b/static/css/styles.css
index f7bc069..210fd2a 100644
--- a/static/css/styles.css
+++ b/static/css/styles.css
@@ -6,7 +6,15 @@ input[type="password"],
 input[type="email"],
 textarea,
 select {
+    width: 100%;
+    padding: 0.75rem;
+    border: 1px solid #d1d5db;
+    border-radius: 0.375rem;
+    background-color: white;
+    font-size: 1rem;
+    line-height: 1.5;
     transition: all 0.3s ease;
+    box-sizing: border-box;
 }
 
 input[type="text"]:focus,
@@ -16,6 +24,44 @@ textarea:focus,
 select:focus {
     border-color: #3b82f6;
     box-shadow: 0 0 0 3px rgba(59, 130, 246, 0.1);
+    outline: none;
+}
+
+/* Django form field specific styling */
+input[id="id_username"],
+input[id="id_password"],
+input[name="username"],
+input[name="password"] {
+    width: 100%;
+    padding: 0.75rem;
+    border: 1px solid #d1d5db;
+    border-radius: 0.375rem;
+    background-color: white;
+    font-size: 1rem;
+    line-height: 1.5;
+    transition: all 